# Step 4: Implementation of “Evidence for the utility of quantum computing before fault tolerance”
paper by Kim Youngseok, Andrew Eddings, et al., 2023, Nature, Vol 618, pg. 501, https://www.nature.com/articles/s41586-023-06096-3

This notebook includes Zero Noise Extrapolation noise mitigation technique to estimate the expectation value of running the algorithm at zero noise. Both polynomial ZNE and exponential ZNE are implemented.

### Setup and Importing Dependencies

This cell imports the necessary libraries and modules for quantum circuit design, execution, data processing, and file handling.


In [50]:
import csv
import numpy as np
from classiq import *
from classiq.execution import ExecutionPreferences
from scipy.optimize import curve_fit

In this cell, we initialize the necessary global variables and constants. We also set the random seed to 64 for consistent testing.

In [66]:
np.random.seed(64)

# circuit variables
TROTTER_STEPS = 4       # number of Trotter steps to be applied
THETA_H = 0             # the rotation angle over the X axis in each Trotter step
SHOTS = 1000            # number of shots for running each ZNE sample
QUBITS = 8              # number of qubits in the register to evolve

# noise simulation variables
NOISE_FACTOR = 0.03     # upper bound of simulated noise
SIMULATED = True        # whether to simulate noise or not [set to False when running on quantum hardware]

# noise mitigation variables
NOISE_AMPLIFICATION_FACTOR = 0      # noise factor for ZNE amplification (overwritten by each ZNE sample)
NOISE_SCALARS_COUNT = 10            # number of ZNE samples
NOISE_SCALAR_FACTOR = 0.1           # bound for sampling noise scalars
NOISE_SCALARS = np.random.uniform(
    -NOISE_SCALAR_FACTOR, NOISE_SCALAR_FACTOR,NOISE_SCALARS_COUNT)   # random sampling of noise scalars
AMPLIFY_NOISE = True                # whether to apply ZNE noise amplification or not

### Pauli-Noise Simulation
Here we simulate the noise that would be present in a quantum processor by inserting Pauli rotations operators with random rotations. This should only be used when running on a simulator.

In [52]:
@qfunc
def simulate_pauli_noise(q: QBit) -> None:
    """ Simulates Pauli noise on a qubit by applying random rotations.

    The function mimics noise that would be present in a quantum processor by applying
    random rotations around the X, Y, and Z axes. This simulation is intended for use with
    quantum simulators and should not be used with actual quantum hardware.

    Args:
        q (QBit): The qubit on which to apply the simulated noise.
    """

    # pauli_rotation(np.random.uniform(0, NOISE_FACTOR), q)
    RX(np.pi * np.random.uniform(0, NOISE_FACTOR), q)
    RY(np.pi * np.random.uniform(0, NOISE_FACTOR), q)
    RZ(np.pi * np.random.uniform(0, NOISE_FACTOR), q)

### Core Quantum Simulation Functions

In this section, we define the functions used to build our quantum circuit. These functions handle the application of Trotterized time evolution, noise amplification, and the setup for getting the expectation value of the circuit.

In [53]:
@qfunc
def noisy_CX(control: QBit, target: QBit) -> None:
    """ Applies a noisy CNOT gate to the specified qubits.

    Args:
        control (QBit): The control qubit.
        target (QBit): The target qubit.
    """

    CX(control, target)

    if SIMULATED:
        simulate_pauli_noise(target)

    if AMPLIFY_NOISE:
        RX(np.pi * NOISE_AMPLIFICATION_FACTOR, target)
        RY(np.pi * NOISE_AMPLIFICATION_FACTOR, target)
        RZ(np.pi * NOISE_AMPLIFICATION_FACTOR, target)

@qfunc
def trotter_step(arr: QArray[QBit]) -> None:
    """ Applies a single Trotter step to the given quantum array.

    Args:
        arr (QArray[QBit]): Quantum array to update.
    """

    apply_to_all(lambda q: RX(THETA_H, q), arr)

    apply_to_all(lambda q: invert(lambda: S(q)), arr)

    for i in range(0, QUBITS-1, 2):
        within_apply(compute=lambda: RY(np.pi/2, arr[i+1]), action=lambda: noisy_CX(arr[i], arr[i+1]))


    apply_to_all(lambda q: invert(lambda: S(q)), arr)

    for i in range(1, QUBITS, 2):
        if i == QUBITS-1:
            within_apply(compute=lambda: RY(np.pi/2, arr[i]), action=lambda: noisy_CX(arr[0], arr[i]))
        else:
            within_apply(compute=lambda: RY(np.pi/2, arr[i+1]), action=lambda: noisy_CX(arr[i], arr[i+1]))

@qfunc
def trotterized_time_evolution(arr: QArray[QBit]) -> None:
    """ Applies Trotterized time evolution to the given quantum array.

    Args:
        arr (QArray[QBit]): Quantum array to evolve.
    """

    for _ in range(TROTTER_STEPS):
      trotter_step(arr)

@qfunc
def main(expectation_value: Output[QBit]) -> None:
    arr = QArray("reg")
    allocate(QUBITS, arr)

    allocate(1, expectation_value)

    within_apply(lambda: hadamard_transform(expectation_value), lambda: control(expectation_value, lambda: trotterized_time_evolution(arr)))


### Measurement and Execution Functions

This cell defines functions for synthesizing and executing the quantum circuit, as well as measuring the expectation value from the results.

In [54]:
def synthesize_execute(shots: int) -> dict:
    """ Synthesizes and executes the main quantum model with the given shots.

    Args:
        shots (int): Number of shots (repetitions).

    Returns:
        dict: Parsed measurement counts.
    """

    quantum_model = set_execution_preferences(
    create_model(main),
    ExecutionPreferences(num_shots=shots),
    )

    quantum_program = synthesize(quantum_model)
    job = execute(quantum_program)
    results = job.result()[0].value.parsed_counts

    return results

def measure_expectation_value(shots: int) -> float:
    """ Measures the expectation value of a quantum circuit based on the measurement
    results using the given number of shots.

    Args:
        shots (int): The number of times to execute the quantum circuit.

    Returns:
        float: The calculated expectation value based on the measurement probabilities.
    """

    result = synthesize_execute(shots)

    if result[0].state["expectation_value"] == 0:
        prob = result[0].shots / shots
    else:
        prob = (shots - result[0].shots) / shots

    return 2*prob - 1

Uncomment and run the following cell to view the visual diagram of the quantum circuit.

In [55]:
# qprog = synthesize(create_model(main))
# show(qprog)

## Classical Post-Processing

This section handles the classical post-processing of measurement data. It includes noise mitigation to estimate the expectation value at zero noise and test cells.


### Noise Mitigation Using Zero Noise Extrapolation (ZNE)
This cell implements zero noise extrapolation by running the circuit multiple times with different noise levels, fitting the results to a model, and then extrapolating to estimate the expected value at zero noise.

In [56]:
def apply_ZNE() -> tuple[float, float]:
    """ Applies Zero Noise Extrapolation (ZNE) to estimate the zero-noise expectation value.

    Measures expectation values at various noise levels, then extrapolates to zero noise using
    both polynomial and exponential fitting methods.

    Returns:
        tuple: Extrapolated zero-noise values from polynomial and exponential fits.
    """

    results = []
    for i in range(len(NOISE_SCALARS)):
        scale_noise(NOISE_SCALARS[i])
        print(f"Sampling {i+1}/{len(NOISE_SCALARS)}...")
        measurement = measure_expectation_value(SHOTS)
        results.append(measurement)

    # Extrapolate to zero noise
    extrapolated_poly_result = extrapolate_poly_zero_noise(results, NOISE_SCALARS)
    extrapolated_exp_result = extrapolate_exp_zero_noise(results, NOISE_SCALARS)
    return extrapolated_poly_result, extrapolated_exp_result

def scale_noise(factor: float) -> None:
    """ Sets the noise amplification factor for the circuit.

    Args:
        factor (float): The noise amplification factor to apply.
    """

    global NOISE_AMPLIFICATION_FACTOR
    NOISE_AMPLIFICATION_FACTOR = factor

def extrapolate_poly_zero_noise(results: list[float], noise_levels: list[float]) -> float:
    """ Fits a polynomial to measurement results and extrapolates to zero noise.

    Args:
        results (list): Measurement results at different noise levels.
        noise_levels (list): Corresponding noise levels.

    Returns:
        float: Extrapolated expectation value at zero noise.
    """

    polynomial = fit_polynomial(noise_levels, results)
    zero_noise_result = evaluate_polynomial(polynomial, 0)
    return zero_noise_result

def fit_polynomial(x: list[float], y: list[float]) -> np.ndarray:
    """ Fits a polynomial to data using least squares method.

    Args:
        x (list): Input data (i.e., noise levels).
        y (list): Output data (i.e., measurement results).

    Returns:
        numpy.ndarray: Coefficients of the fitted polynomial.
    """

    polynomial = np.polyfit(x, y, deg=2)
    return polynomial

def evaluate_polynomial(polynomial: np.ndarray, x: float) -> float:
    """ Evaluates a polynomial at the given point.

    Args:
        polynomial (np.ndarray): Coefficients of the polynomial.
        x (float): The point at which to evaluate the polynomial.

    Returns:
        float: The result of the polynomial evaluation.
    """

    result = np.polyval(polynomial, x)
    return result

def extrapolate_exp_zero_noise(results: list[float], noise_levels: list[float]) -> float:
    """ Fits an exponential model to results and extrapolates to zero noise.

    Args:
        results (list): Measurement results at different noise levels.
        noise_levels (list): Corresponding noise levels.

    Returns:
        float: Extrapolated expectation value at zero noise.
    """

    params = fit_exponential(noise_levels, results)
    return params[0]

def exponential_model(x: float, a: float, b: float) -> float:
    """ Exponential function for fitting.

    Args:
        x (float): Noise level.
        a (float): Amplitude of the exponential function.
        b (float): Rate of exponential growth.

    Returns:
        float: The result of the exponential function.
    """

    return a * np.exp(b * x)

def fit_exponential(x: list[float], y: list[float]) -> tuple[float, float]:
    """ Fits an exponential model to data using curve fitting.

    Args:
        x (list): Input data (i.e., noise levels).
        y (list): Output data (i.e., measurement results).

    Returns:
        tuple: Fitted parameters (amplitude and rate) of the exponential model.
    """

    params, _ = curve_fit(exponential_model, x, y)
    return params

### Saving Results to CSV

This cell appends the variables and processed results to a CSV file for future analysis.


In [59]:
def write_results(filename: str, ZNE_result: tuple[float, float], unmitigated_result: float) -> None:
    row = [
        QUBITS,
        SHOTS,
        NOISE_FACTOR,
        TROTTER_STEPS,
        NOISE_SCALAR_FACTOR,
        NOISE_SCALARS_COUNT,
        ZNE_result[0],
        ZNE_result[1],
        unmitigated_result,
    ]

    with open(filename, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(row)

### Test with Constant $\theta_h$

When $\theta_h$ is set to $0$, the $RX$ gates have no effect, so the expectation value should be 1. We can now run the algorithm with noise mitigation and compare the results to the ideal value of 1.

> Note that for ZNE to be most effective, multiple noise-scaled runs of the algorithm are required. However, running thousands or even hundreds of thousands of samples on a simulator is time-consuming, so we've reduced the number of samples and shots to speed up the process, which may impact accuracy.

In [67]:
THETA_H = 0

ZNE_result = apply_ZNE()

print("[Polynomial Fit] Expectation value:", ZNE_result[0])
print("[Exponential Fit] Expectation value:", ZNE_result[1])


print("Running unmitigated control.")
scale_noise(0)
unmitigated_result = measure_expectation_value(SHOTS)
print("[Unmitigated] Expectation value:", unmitigated_result)

write_results("test1.csv", ZNE_result, unmitigated_result)


Sampling 1/10...
-0.024180294913245984
Sampling 2/10...
0.013419633435922812
Sampling 3/10...
0.019118502531424372
Sampling 4/10...
-0.010028220228909479
Sampling 5/10...
-0.008596034921056275
Sampling 6/10...
-0.0376698004249209
Sampling 7/10...
-0.03733252776048593
Sampling 8/10...
-0.04102810779968391
Sampling 9/10...
0.006154492247125276
Sampling 10/10...
-0.02075559568011623
[Polynomial Fit] Expectation value: 0.9590252681741838
[Exponential Fit] Expectation value: 0.6767049382934467
Running unmitigated control.
0
[Unmitigated] Expectation value: 0.6759999999999999


### Test with Changing $\theta_h$

Now we run multiple tests each with a different value of $\theta_h$.

In [62]:
angles = [0, np.pi/8, np.pi/4, 3*np.pi/8, np.pi/2]

for angle in angles:
    THETA_H = angle
    ZNE_result = apply_ZNE()

    print("[Polynomial Fit] Expectation value:", ZNE_result[0])
    print("[Exponential Fit] Expectation value:", ZNE_result[1])

    print("Running unmitigated control.")
    scale_noise(0)
    unmitigated_result = measure_expectation_value(SHOTS)
    print("[Unmitigated] Expectation value:", unmitigated_result)

    write_results("test2.csv", ZNE_result, unmitigated_result)

Sampling 1/10...
-0.024180294913245984
Sampling 2/10...
0.013419633435922812
Sampling 3/10...
0.019118502531424372
Sampling 4/10...
-0.010028220228909479
Sampling 5/10...
-0.008596034921056275
Sampling 6/10...
-0.0376698004249209
Sampling 7/10...
-0.03733252776048593
Sampling 8/10...
-0.04102810779968391
Sampling 9/10...
0.006154492247125276
Sampling 10/10...
-0.02075559568011623
[Polynomial Fit] Expectation value: 0.5925317790538206
[Exponential Fit] Expectation value: 0.040864404762594335
Running unmitigated control.
0
[Unmitigated] Expectation value: -0.06000000000000005
Sampling 1/10...
-0.024180294913245984
Sampling 2/10...
0.013419633435922812
Sampling 3/10...
0.019118502531424372
Sampling 4/10...
-0.010028220228909479
Sampling 5/10...
-0.008596034921056275
Sampling 6/10...
-0.0376698004249209
Sampling 7/10...
-0.03733252776048593
Sampling 8/10...
-0.04102810779968391
Sampling 9/10...
0.006154492247125276
Sampling 10/10...
-0.02075559568011623
[Polynomial Fit] Expectation value: 